In [ ]:
import openpyxl
import pandas as pd

In [21]:
# 勤務時間を計算する関数 ※勤務時間カラムを引数に設定する。
def to_hour(df):
    hour = int(df_name_filtered['勤務時間'].sum()/60)
    minute = df_name_filtered['勤務時間'].sum()%60
    total_working_hour = f'{hour}:{minute}'
    return total_working_hour

wb_members = openpyxl.load_workbook('従業員集約.xlsx', data_only=True)
names = wb_members.sheetnames

data = []

for name in names:
    df_name = pd.read_excel('従業員集約.xlsx', sheet_name=name)

    start_date = '2021-10-1'
    end_date = '2021-10-31'

    df_name_filtered = df_name[(df_name['日付'] >= start_date) & (df_name['日付'] <= end_date)]
    total_working_hour = df_name_filtered['勤務時間'].sum()
    working_days = len(df_name_filtered[df_name_filtered['勤務時間'] != 0])
    transportation_expense = df_name_filtered['交通費'].sum()
    n_midnight_work = len(df_name_filtered[df_name_filtered['深夜勤務'] == '◯'])

    datum = {
        '氏名':name,
        '総労働時間':total_working_hour,
        '勤務日数':working_days,
        '交通費':transportation_expense,
        '深夜勤務日数':n_midnight_work,
    }
    data.append(datum)

df_summary = pd.DataFrame(data)

wb_summary = openpyxl.Workbook()
wb_summary.save('集計.xlsx')

wb_summary = openpyxl.load_workbook('集計.xlsx', data_only=True)
sheet_name = '2021年10月分'
wb_summary.create_sheet(title = sheet_name)
ws_summary = wb_summary[sheet_name]

ws_summary['A1'].value = '氏名'
ws_summary['B1'].value = '総労働時間'
ws_summary['C1'].value = '勤務日数'
ws_summary['D1'].value = '交通費'
ws_summary['E1'].value = '深夜勤務日数'

ws_summary.column_dimensions['A'].width = 10
ws_summary.column_dimensions['B'].width = 10
ws_summary.column_dimensions['C'].width = 10
ws_summary.column_dimensions['D'].width = 10
ws_summary.column_dimensions['E'].width = 10

for index, row  in df_summary.iterrows():
    ws_summary.cell(row = index+2, column = 1).value = row['氏名']
    ws_summary.cell(row = index+2, column = 2).number_format = '[hh]:mm'
    ws_summary.cell(row = index+2, column = 2).value = row['総労働時間']
    ws_summary.cell(row = index+2, column = 3).value = row['勤務日数']
    ws_summary.cell(row = index+2, column = 4).value = row['交通費']
    ws_summary.cell(row = index+2, column = 5).value = row['深夜勤務日数']

wb_summary.save('集計.xlsx')